In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [109]:
#loading data
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [110]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [111]:
test_ids = test.PassengerId #or test['PassengerId']
y = train.Survived

In [112]:
#Convert text label samples to numerical format
from sklearn import preprocessing
text2num = preprocessing.LabelEncoder()
textCols= ['Sex'] #Can add more cols to modify

for col in  textCols:
    train[col] = text2num.fit_transform(train[col])
    test[col] = text2num.transform(test[col])
    
    print(text2num.classes_)
    
train.head(5)

['female' 'male']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [113]:
#Drop unwanted cols
neededTrainCols = train.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin',	'Embarked'], axis=1)
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin',	'Embarked'], axis=1)

#y.head(5)
X_train.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare
501,3,0,21.0,0,0,7.7500
704,3,1,26.0,1,0,7.8542
202,3,1,34.0,0,0,6.4958
588,3,1,22.0,0,0,8.0500
523,1,0,44.0,0,1,57.9792


In [114]:
#Add missing values
for col in neededTrainCols:
    neededTrainCols[col] = neededTrainCols[col].fillna(neededTrainCols[col].mean())
    #neededTrainCols[col].fillna(neededTrainCols[col].mean(), inplace=True)
    
for col in test:
    test[col] = test[col].fillna(test[col].mean())

In [117]:
#Creating model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = neededTrainCols
#y = 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=38)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
prediction = clf.predict(X_val)

accuracy_score(y_val, prediction)

0.8156424581005587

In [121]:
submission = clf.predict(test)
df = pd.DataFrame({'PassengerId' : test_ids.values,
                   'Survived' : submission
                  })

df.to_csv('submission.csv', index=False)